In [1]:
from IPython.display import display, Math, HTML, Image
import orbits 

#%config InlineBackend.figure_formats = ['svg']
from astropy.visualization import quantity_support, astropy_mpl_style
import astropy.units as u
from astropy.coordinates import SkyCoord
quantity_support()  
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib as mpl
import matplotlib_inline
import gaia
import numpy as np
from scipy import stats
from geom import angular_dist, deg2rad, rad2deg
from astropy.stats import sigma_clip

#matplotlib_inline.backend_inline.set_matplotlib_formats('svg')
mpl.style.use('default')
mpl.style.use('tableau-colorblind10')
mpl.style.use('./MNRAS_Style.mplstyle')
mpl.rcParams['figure.figsize'] = [12/3, 12/3]
#mpl.rcParams['figure.dpi'] = 72

# in points - start with the body text size and play around
SMALL_SIZE = 9
MEDIUM_SIZE = 10
BIGGER_SIZE = 9

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)

display(HTML("<h1>NGC 6397</h1>"))
# display(Image(url='https://upload.wikimedia.org/wikipedia/commons/9/99/P0321a.jpg', width=1024))

display(Math('\\text{{RA}} = {}^{{\\circ}}'.format(orbits.RA)))
display(Math('\\text{{DEC}} = {}^{{\\circ}}'.format(orbits.DEC)))
display(Math('\\mu_\\alpha \\cos\\delta = {}^{{\\circ}}'.format(orbits.PMRA)))
display(Math('\\delta = {}^{{\\circ}}'.format(orbits.PMDEC)))
display(Math('R_\\odot = {}\\text{{ kpc}}'.format(orbits.RSUN)))
display(Math('\\sigma = {}\\text{{ mas/yr}}'.format(orbits.SIGMA)))




#nbody = np.loadtxt('2023-11-05/ngc6397.out')
nbody = np.loadtxt('2024-04-07/ngc6397.out')
nbody_irr = np.loadtxt('2024-04-28/ngc6397.out')
rtide = np.loadtxt('2024-04-07/boundm')[:,1]

display(Math('N_\\text{{initial}} = {}'.format(len(gaia.data))))

# Clean out bad data
clean = (gaia.data['bp_rp'] != 0) 

display(HTML('Cleaning for zero BP-RP'))
display(Math('N_\\text{{clean}} = {}'.format(np.sum(clean))))



# Perform a proper motion cut
cut = clean


cut &= np.abs(gaia.data['pmra'] - orbits.PMRA) < (2 * gaia.data['pmra_error'])
cut &= np.abs(gaia.data['pmdec'] - orbits.PMDEC) < (2 * gaia.data['pmdec_error'])

display(HTML('Proper motion cut'))
display(Math('\\lvert \\mu_\\alpha \\cos\\delta_i - \\mu_\\alpha \\cos\\delta \\rvert < 2 \\epsilon_{{\\mu_\\alpha \\cos\\delta_i}}'))
display(Math('\\lvert \\mu_{\\delta_i }- \\mu_{\\delta} \\rvert < 2 \\epsilon_{{\\mu_{\\delta_i}}}'))
display(Math('N_\\text{{pm}} = {}'.format(np.sum(cut))))

distances = angular_dist(gaia.data['ra'], gaia.data['dec'], orbits.RA, orbits.DEC)

def draw_pm(save=False, outer=None):
    # Draw a proper motion diagram.
    fig, ax = plt.subplots(ncols=1)
    fig.tight_layout()
    ax.set(
        xlim=[orbits.PMRA-15, orbits.PMRA+15],
        ylim=[orbits.PMDEC-15, orbits.PMDEC+15]
    )
    ax.set_xlabel(r'$\mu_{\alpha} \cos \delta$ [mas/yr]')
    ax.set_ylabel(r'$\mu_{\delta}$ [mas/yr]')

    inside = distances < deg2rad(3.0)
    
    # Stars outside the proper motion space
    ax.scatter(gaia.data['pmra'][bg&inside], gaia.data['pmdec'][bg&inside], s=0.1, marker='.')
    
    # Stars inside the proper motion space
    ax.scatter(gaia.data['pmra'][cut&inside], gaia.data['pmdec'][cut&inside], s=0.1, marker='.')

    # Other stars
    if outer is not None:
        ax.scatter(gaia.data['pmra'][outer], gaia.data['pmdec'][outer], s=0.3, color='black', marker=(5,1))
    
    if save:
        fig.savefig('pm.png')

#draw_pm()


# Perform a statistical selection
chi2 = 5
cut &= (gaia.data['pmra'] - orbits.PMRA)**2 / (orbits.SIGMA**2 + gaia.data['pmra_error']**2) + \
       (gaia.data['pmdec'] - orbits.PMDEC)**2 / (orbits.SIGMA**2 + gaia.data['pmdec_error']**2) <= chi2


display(HTML('Statistical criterion cut'))
display(Math('''\\chi^2 = \\frac{(\\mu_\\alpha \\cos\\delta_i - \\mu_\\alpha \\cos\\delta)^2}{(\\sigma^2 + \\epsilon_{{\\mu_\\alpha \\cos\\delta}})^2} + \\
                \\frac{(\\mu_{\\delta_i} - \\mu_{\\delta_i})^2}{(\\sigma^2 + \\epsilon_{{\\mu_{\\delta_i}}})^2} \\le 5''')) 
display(Math('N_{{\\chi^2}} = {}'.format(np.sum(cut))))

pm_cut = cut.copy()
#draw_pm()

# Parallax cut
print(np.sum(cut & (gaia.data['parallax'] < 0)))
px_cut = np.divide(
    (gaia.data['parallax'] - orbits.PARALLAX)**2,
    gaia.data['parallax_error']**2,
    out=np.zeros_like(gaia.data['parallax']),
    where=(gaia.data['parallax'] > 0) & (gaia.data['parallax_error'] != 0))
cut &= px_cut < 3
display(HTML('Parallax cut = {}'.format(np.sum(cut))))
pm_cut = cut.copy()

# Perform a colour-magnitude cut
def cmcut(cut):
    bins = np.arange(14, 22, 0.1)

    y = np.ma.array(gaia.data['bp_rp'])
    inv_mask = np.full(len(y), False)
    for b in bins:
        band = cut & (gaia.data['phot_g_mean_mag'] >= b) & (gaia.data['phot_g_mean_mag'] < b + 0.1)
        y.mask = ~band
        clipped = sigma_clip(y, sigma=2, maxiters=10)
        inv_mask |= ~clipped.mask
    
    return inv_mask

#cut &= cmcut(cut)


# Seperate the cluster into inside 0.5 degree radius and outside.
inside = distances < deg2rad(0.5)

# Find all of the faint stars (mag > 19) 
faint = gaia.data['phot_g_mean_mag'] > 19

# Of the stars outside the 0.5 deg radius, determine if
# the angular separation is of the same distribution.
g1 = cut & ~inside & faint
g2 = cut & ~inside & ~faint
result = stats.kstest(distances[g1], distances[g2])

# If reasonably certain the two groups are different,
# throw away the faint ones.
if result.pvalue < 0.05:
    display(HTML("Throwing away the faint ones."))
    cut &= ~g1

if np.sum(g2) == 0:
    display(HTML('<strong>Oh no there are no bright stars outside 0.5 deg!</strong>'))


# Throw away stars RA<260
cut &= gaia.data['ra']>260

display(HTML('Final selection'))
display(Math('N_\\text{{final}} = {}'.format(np.sum(cut))))
bg = (gaia.data['bp_rp'] != 0) & ~cut
draw_pm(True)


# Draw a colour-magnitude diagram
inside = distances < deg2rad(3.0)
bg = pm_cut & ~cut

fig, ax = plt.subplots(ncols=1)
fig.tight_layout()
ax.invert_yaxis()
ax.set_xlabel('$(BP-RP)_{Gaia}$ [mag]')
ax.set_ylabel('$G_{Gaia}$ [mag]')
ax.set(
    #title=r'Colour-magnitude Diagram',
    xlim=[0.0, 3.0],
    ylim=[21, 15]
)


ax.scatter(gaia.data['bp_rp'][bg&inside], gaia.data['phot_g_mean_mag'][bg&inside], s=0.1)
ax.scatter(gaia.data['bp_rp'][cut&inside], gaia.data['phot_g_mean_mag'][cut&inside], s=0.1)
ax.hlines(19, 0, 3, colors='black', linestyles='dashed')

fig.savefig('cm.png')



# Plot the spatial coordinates
fig, ax = plt.subplots(ncols=1)
ax.set_xlabel('RA')
ax.set_ylabel('DEC')
ax.set_title('Celestial coordinates - Gaia')
ax.scatter(gaia.data['ra'][cut], gaia.data['dec'][cut], color='b', marker='x', s=1)
ax.add_patch(plt.Circle((orbits.RA, orbits.DEC), 2.778, color='g', fill=False, ls='--'))
ax.add_patch(plt.Circle((orbits.RA, orbits.DEC), 2*2.778, color='r', fill=False, ls='--'))
ax.add_patch(plt.Circle((orbits.RA, orbits.DEC), 11.111, color='b', fill=False, ls='--'))
'''
fig, ax = plt.subplots(ncols=1)

ax.set(
    xlim=[245,285],
    ylim=[-65,-40]
)
ax.set_xlabel('RA')
ax.set_ylabel('DEC')
ax.set_title('Celestial coordinates - NBODY')
ax.scatter(nbody[:,1], nbody[:,2], color='g', marker='.', s=10)
'''

gaia_sb_scale = 15.35

def bg():
    ralim = (270, 295)
    declim = (-58, -56)
    bgdens = (ralim[0] < gaia.data['ra']) & (gaia.data['ra'] < ralim[1])
    bgdens &= (declim[0] < gaia.data['dec']) & (gaia.data['dec']< declim[1])
    area = np.deg2rad(ralim[1] - ralim[0]) * np.deg2rad(declim[1] - declim[0]) * np.cos(np.deg2rad(declim[1] + declim[0]) / 2.0)
    area *= 206264.806**2
    return np.sum(cut & bgdens), area

def background_sb():
    nbbg, bgarea = bg()
    n_area = nbbg / bgarea 
    print(f"{n_area} arcsec^{-2}")
    nsq_area = np.sqrt(nbbg) / bgarea 
    y = -2.5 * np.log10(n_area)
    error = -2.5 * np.log10(n_area+nsq_area) - y
    y += gaia_sb_scale
    return y, np.abs(error)
    
bgsb, bgsb_err = background_sb()
print(f"Background SB {bgsb} +- {bgsb_err}")

def subtract(brightness):
    bg = 10**(bgsb / -2.5)
    n_area = 10**(brightness / -2.5)
    #print(bg, n_area, n_area - bg)
    return -2.5 * np.log10(n_area - bg)
        
    
def surface_density(d, ax, label, scale=17, lw=1, fmt='', color=None, nbins=50):
    logmin = np.log10(np.amin(d))
    logmax = np.log10(np.amax(d))
    
    logbins = [0] + np.logspace(logmin, logmax, nbins)
    counts, edges = np.histogram(d, bins=logbins)
    cut = counts > 0

    # Divide the sum in each bin by the area of the annulus
    deg = edges[1:] / 3600
    left = np.maximum(orbits.RA - deg, 260)
    right = orbits.RA + deg
    top = np.minimum(orbits.DEC + deg, -50)
    bottom = np.maximum(orbits.DEC - deg, -56)
    sqr = (2*deg)**2
    rec = (right-left)*(top-bottom)
    f = rec/sqr
    print(f)
    
    area = np.pi * (edges[1:]**2 - edges[:-1]**2) * f
    
    #area = np.array([
    #  np.pi * (bins[i+1]**2 - r**2)
    #  for i,r in enumerate(r_b)
    #])[cut]
    
    n_area = (counts / area)[cut]
    nsq_area = (np.sqrt(counts) / area)[cut]
    y = -2.5 * np.log10(n_area)
    error = -2.5 * np.log10(n_area+nsq_area) - y
    y += scale
    r_b = edges[:-1][cut]

    #print(n_area)
    ax.errorbar(r_b, y, yerr=np.abs(error), label=label, lw=lw, elinewidth=1, fmt=fmt, color=color)
    return (np.log10(r_b), y, np.abs(error))

'''
def centre_dist(ra, dec):
    return 3600.0 * np.sqrt((ra-orbits.RA)**2 * np.cos(orbits.DEC/180.0*np.pi)**2+(dec-orbits.DEC)**2) 
'''

        
def plot_surface_density(cut, name):
    bright = gaia.data['phot_g_mean_mag'] < 19
    
    # Stitch with Trager et al.
    from geom import rad2arcsec
    trager = np.loadtxt('ngc6397_trager.dat')
    noyola = np.loadtxt('n6397_SB_noyola.txt')
    
    fig, ax = plt.subplots(ncols=1)
    fig.tight_layout()
    
    #ax.set_title('Surface Brightness (Bovy)')
    ax.set_xlabel(r'Radius [arcsec]')
    ax.set_ylabel(r'Surface Brightness [mag arcsec$^{-2}$]')
    ax.set_xscale("log")
    #ax.set_yscale("log")
    ax.grid()
    ax.invert_yaxis()
    
    ax.set(
        ylim=[38, 10],
        xlim=[0.5, 45000]
    )
    
    # ax.set(title="Surface Density")
    
    # Plot tager
    # ax.plot(10**trager[:,0], trager[:,2], label='Trager')
    # clean
    trager = np.loadtxt('ngc6397_trager.dat')
    trager = trager[(np.abs(trager[:,3]) < 1) & (trager[:,0] >= 1.7) & (trager[:,0] < 2.6)]
    groups = np.split(trager, np.unique(trager[:,0], return_index=True)[1][1:])
    maxerr = [np.abs(a[:,3]).argmax(axis=0) for a in groups]
    trager = np.array([g[m] for g,m in zip(groups, maxerr)])
    
    rem = [(trager[1:,0] - trager[:-1,0]) < 0.03]
    while np.any(rem):
        i = 1 + np.argmax(rem)
        trager = np.delete(trager, i, axis=0)
        rem = [(trager[1:,0] - trager[:-1,0]) < 0.03]
    
    trager_err = np.abs(trager[:,3])
    cut_trager = (trager_err < 1) & (trager[:,0] > 1.75) & (trager[:,0] < 2.6)
    ax.errorbar(10**trager[:,0][cut_trager], trager[:,2][cut_trager], yerr=trager_err[cut_trager], label='Trager et al. (1995)')
    #ax.plot(10**trager[:,0][cut_trager], 10**(-0.4*trager[:,2][cut_trager]), label='Trager')
    
    
    # Plot Noyola
    noyola_err = np.abs(noyola[:,2])
    cut_noyola = (noyola_err < 1)
    ax.errorbar(10**noyola[:,0][cut_noyola], noyola[:,1][cut_noyola], yerr=noyola_err[cut_noyola], label='Noyola \\& Gebhardt (2006)')
    #ax.plot(10**noyola[:,0][cut_noyola], 10**(-0.4*noyola[:,1][cut_noyola]), label='Noyola')
    
    # Stitch Gaia
    central = (-56 <= gaia.data['dec']) & (gaia.data['dec'] <= -50)
    gaia_cut = gaia.data[cut & bright & central]
    #arcsec_distance = rad2arcsec(angular_dist(gaia_cut['ra'], gaia_cut['dec'], orbits.RA, orbits.DEC))
    #arcsec_distance = 3600.0 * np.sqrt((gaia_cut['ra']-orbits.RA)**2 * np.cos(orbits.DEC/180.0*np.pi)**2+(gaia_cut['dec']-orbits.DEC)**2) 
    
    arcsec_distance = SkyCoord(gaia_cut['ra'] * u.deg, gaia_cut['dec'] * u.deg).separation(
        SkyCoord(orbits.RA * u.deg, orbits.DEC * u.deg)).to_value(u.arcsec)
    
    gaia_r, gaia_dens, gaia_err = surface_density(arcsec_distance[arcsec_distance > 100], ax, 'Gaia DR3 shifted', scale=gaia_sb_scale, nbins=30)
    
    sb_table = np.array([noyola[:,0][cut_noyola], noyola[:,1][cut_noyola], np.abs(noyola[:,2][cut_noyola]), np.full(np.sum(cut_noyola), 1)]).T
    sb_table = np.append(sb_table, np.array([trager[:,0][cut_trager], trager[:,2][cut_trager], np.abs(trager[:,3][cut_trager]), np.full(np.sum(cut_trager), 2)]).T, axis=0)
    sb_table = np.append(sb_table, np.array([gaia_r, gaia_dens, gaia_err, np.full(len(gaia_r), 3)]).T, axis=0)
    np.savetxt('sb.txt', sb_table,fmt=['%.3f', '%.2f', '%.3f', '%d'], delimiter=' & ', newline='\\\\\n')
    # Get nbody form
    #m80 = (nbody[:,7] > 0.8) & (nbody[:,8] < 10)
    
    #nbdis = rad2arcsec(angular_dist(nbody[:,1][m80], nbody[:,2][m80], orbits.RA, orbits.DEC))
    
    #surface_density(nbdis, ax, 'NBODY 4gyr $+16$', fmt='g-', scale=16, lw=3)
    lc = ax.hlines(bgsb, 2* 10**3, 4.5 * 10**4, colors='black', linestyles='dashed', label='Background surface density')
    
    ax.legend(loc='lower left')
    fig.savefig(f'{name}.png')
        
    lc.remove()

    # Get nbody form
    def get_nbdis(nb):
        m80 = (nb[:,7] > 0.8) & (nb[:,8] < 10)
        return 3600.0 * np.sqrt((nb[:,1][m80]-orbits.RA)**2 * np.cos(orbits.DEC/180.0*np.pi)**2+(nb[:,2][m80]-orbits.DEC)**2) 

    nbdis = get_nbdis(nbody)
    nbdis_irr = get_nbdis(nbody_irr)
    surface_density(nbdis, ax, 'NBODY (Bovy) shifted', scale=13.2, lw=3, fmt='x', color='#595959')
    surface_density(nbdis_irr, ax, 'NBODY (Irrgang) shifted', scale=13.2, lw=3, fmt='+', color='#5F9ED1')
    ax.vlines(1/(rtide[-1:]*3600), ymin=10, ymax=36, linestyles='dashed', colors='k')
    ax.legend(loc='lower left')
    fig.savefig(f'{name}-nb.png')

    fig, ax = plt.subplots()
    fig.tight_layout()
    ax.set_xscale('log')
    ax.grid()
    ax.set(
        xlim=[80, 80000],
        ylabel=r'log Surface Density [mag arcsec$^{-2}$]',
        xlabel=r'Radius [arcsec]')
    ax.errorbar(10**gaia_r, -0.4*(gaia_dens-gaia_sb_scale), yerr=np.abs(gaia_err), elinewidth=1)
    ax.errorbar(10**gaia_r, -0.4*(subtract(gaia_dens)-gaia_sb_scale), yerr=np.abs(gaia_err), elinewidth=1)
    ax.hlines(-0.4*(bgsb-gaia_sb_scale), 2* 10**3, 5 * 10**4, colors='black', linestyles='dashed', label='Background surface density')
    fig.savefig('standalone-profile.png')    
    
    fig, axes = plt.subplots(nrows=2, figsize=(12/3, 24/3), sharex=True)
    fig.tight_layout()
    
    for ax in axes:
        ax.set_xscale("log")
        ax.grid()
        #ax.invert_yaxis()
        
    axes[0].set(
        ylabel=r'log Surface Density [mag arcsec$^{-2}$]',
        #ylim=[38, 10],
        xlim=[0.5, 45000]
    )
    axes[1].set(
        ylabel=r'log Surface Density [mag arcsec$^{-2}$]',
        xlabel=r'Radius [arcsec]',
        #ylim=[38, 10],
        xlim=[0.5, 45000]
    )


    axes[0].errorbar(10**trager[:,0][cut_trager], -0.4*(trager[:,2][cut_trager]), yerr=trager_err[cut_trager], label='Trager et al. (1995)')
    axes[0].errorbar(10**noyola[:,0][cut_noyola], -0.4*(noyola[:,1][cut_noyola]), yerr=noyola_err[cut_noyola], label='Noyola \\& Gebhardt (2006)')
    axes[0].errorbar(10**gaia_r, -0.4*(gaia_dens), yerr=np.abs(gaia_err), label='Gaia DR3 shifted', elinewidth=1)
    axes[0].hlines(-0.4*bgsb, 2* 10**3, 4.5 * 10**4, colors='black', linestyles='dashed', label='Background surface density')

    gaia_ok = gaia_dens < bgsb
    axes[1].errorbar(10**trager[:,0][cut_trager], -0.4*subtract(trager[:,2][cut_trager]), yerr=trager_err[cut_trager], label='Trager et al. (1995)')
    axes[1].errorbar(10**noyola[:,0][cut_noyola], -0.4*subtract(noyola[:,1][cut_noyola]), yerr=noyola_err[cut_noyola], label='Noyola \\& Gebhardt (2006)')
    axes[1].errorbar(10**gaia_r[gaia_ok], -0.4*subtract(gaia_dens[gaia_ok]), yerr=np.abs(gaia_err[gaia_ok]), label='Gaia DR3 shifted', elinewidth=1)
    axes[1].hlines(-0.4*bgsb, 2* 10**3, 4.5 * 10**4, colors='black', linestyles='dashed', label='Background surface density')
    axes[1].legend(loc='lower left')
    
    fig.savefig(f'{name}-subtract.png')

    '''

    # Plot subtracted background brightness
    fig, ax = plt.subplots(ncols=1)
    fig.tight_layout()
    
    ax.set_xlabel(r'Radius [arcsec]')
    ax.set_ylabel(r'Surface Brightness [mag arcsec$^{-2}$]')
    ax.set_xscale("log")
    #ax.set_yscale("log")
    ax.grid()
    ax.invert_yaxis()
    
    ax.set(
        ylim=[38, 10],
        xlim=[0.5, 45000]
    )

    ax.errorbar(10**trager[:,0][cut_trager], subtract(trager[:,2][cut_trager]), yerr=trager_err[cut_trager], label='Trager et al. (1995)')
    ax.errorbar(10**noyola[:,0][cut_noyola], subtract(noyola[:,1][cut_noyola]), yerr=noyola_err[cut_noyola], label='Noyola \\& Gebhardt (2006)')
    ax.errorbar(10**gaia_r, subtract(gaia_dens), yerr=np.abs(gaia_err), label='Gaia DR3 shifted', elinewidth=1)
    #surface_density(arcsec_distance[arcsec_distance > 300], ax, 'Gaia DR3 shifted', scale=gaia_sb_scale, nbins=25, sub=True)
    ax.hlines(bgsb, 2* 10**3, 4.5 * 10**4, colors='black', linestyles='dashed', label='Background surface density')
    
    ax.legend(loc='lower left')
    fig.savefig(f'{name}-subtract.png')
    '''
    
    pass


plot_surface_density(cut, 'sb')

'''
fig, ax = plt.subplots(ncols=1)
fig.tight_layout()
ax.invert_yaxis()
ax.set_xlabel('$(BP-RP)_{Gaia}$ [mag]')
ax.set_ylabel('$G_{Gaia}$ [mag]')
ax.set(
    #title=r'Colour-magnitude Diagram',
    xlim=[0.0, 3.0],
    ylim=[21, 15]
)

# remove the outer
outer = cut & (angular_dist(gaia.data['ra'], gaia.data['dec']).to_value(u.arcsec) > 3000)
cut = cut & ~outer
bg = pm_cut & ~cut


ax.scatter(gaia.data['bp_rp'][bg], gaia.data['phot_g_mean_mag'][bg], s=0.1)
ax.scatter(gaia.data['bp_rp'][cut], gaia.data['phot_g_mean_mag'][cut], s=0.1)
#ax.scatter(gaia.data['bp_rp'][outer], gaia.data['phot_g_mean_mag'][outer], color='black', marker=(5, 1), s=0.3)

#fig.savefig('cm-with-flattening.png')

bg = (gaia.data['bp_rp'] != 0) & ~cut
draw_pm()#outer=outer)


plot_surface_density(cut, 'sb2')
'''



pass

KeyboardInterrupt: 

In [ ]:

from astropy.stats import sigma_clip


def fit_cm():
    colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
    fig, ax = plt.subplots(ncols=1)
    fig.tight_layout()

    ax.invert_yaxis()
    ax.set_xlabel('Gaia BP-RP Colour')
    ax.set_ylabel('Gaia G absolute space')
    ax.set(
        #title=r'Colour-magnitude Diagram',
        xlim=[0.0, 3.0],
        ylim=[21, 15]
    )

    fig1, ax1 = plt.subplots(ncols=1)
    fig1.tight_layout()
    #ax1.set(
    #    xlim=[245,285],
    #    ylim=[-65,-40]
    #)
    ax1.set_xlabel('RA')
    ax1.set_ylabel('DEC')

    means = []
    stdevs = []
    bins = np.arange(14, 19, 0.2)

    x = np.ma.array(gaia.data['phot_g_mean_mag'], mask=cut)
    y = np.ma.array(gaia.data['bp_rp'], mask=cut)
    mask = np.full(len(y), True)
    for b in bins:
        band = cut & (gaia.data['phot_g_mean_mag'] >= b) & (gaia.data['phot_g_mean_mag'] < b + 0.1)
        x.mask = band
        y.mask = band
        clipped = sigma_clip(y, sigma=2, maxiters=10)
        
        ax.scatter(y[~clipped.mask], x[~clipped.mask], s=0.1, color=colors[1])
        ax.scatter(y[clipped.mask], x[clipped.mask], s=0.1, color=colors[0])

        ax1.scatter(gaia.data['ra'][~clipped.mask], gaia.data['dec'][~clipped.mask], color=colors[1], marker='.', s=1)
        ax1.scatter(gaia.data['ra'][clipped.mask], gaia.data['dec'][clipped.mask], color=colors[0], marker='.', s=1)
        
        mask &= clipped.mask
        
    #ax.errorbar(means, bins, xerr=stdevs)
    fig.savefig('cm-binned.png')
    print(np.sum(mask))

fit_cm()

In [ ]:

"""
def get_cen(nb):
    cut = nb[:2000,1:3]
    dis = np.sum((cut[:,np.newaxis] - nb[:,1:3])**2, axis=2)
    dens = 1.0 / np.amin(dis, where=dis>0, axis=1, initial=1e10)**3
    weight = np.sum(dens)
    return np.sum(cut * dens[:,np.newaxis], axis=0) / weight
    
    
nb8_cen = get_cen(nbody)
print(nb8_cen)
"""
#nb8_cen = [270.164528, -54.355833]
#nb8_cen = [265.175385   ,-53.674335 ]

nb8_cen = [orbits.RA, orbits.DEC]

In [ ]:
# Vel. disp.
from astropy import units as u
import veldis
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)

def plot_velocity_dispersion(save):

    
    def get_veldis(vel, dx, logbins=None):
        if logbins is None:
            logbins = np.logspace(1.2, 3.2, num=12)
            
        #hi = 10**(np.log10(logbins[-1])+0.1)
        hi = 5000
        
        nbinner = (dx < hi)
        nb_v_mean = np.mean(vel[nbinner])
        outlier = ~nbinner | (np.abs(vel - nb_v_mean) > 25)
        print(f"total {len(vel)} inner {np.sum(nbinner)} outlier {np.sum(outlier)}")
        
        hist = []
        bins = []
        errs = []
        ns = 0
        r_0 = 0
        for i,r in enumerate(logbins):
            r_1 = 10**(np.log10(r)+0.1)
            
            bds = ~outlier & (dx > r_0) & (dx < r_1)
            n = np.sum(bds)
            #if n == 0:
            #    continue
            s = np.sqrt(1/n * np.sum((vel[bds] - nb_v_mean)**2.0))
            ns += n
            error = s / np.sqrt(n)
    
            hist.append(s)
            bins.append(r)
            errs.append(error)
            r_0 = r_1
    
        return np.array(hist), np.array(errs), np.array(bins)
    
    
    def tangent_to_velocity(mas):
        return mas.value/3600/180*np.pi * orbits.RSUN * 3.085677E16 / 3.15576e7 / 1000
    
    
    def plot_veldis_nb(ax, label, nb, type, color, fmt='--*'):
        #ax1, ax2 = axes
        
        m80 = (nb[:,8] < 10) & (nb[:,7] > 0.8) 
        bright = nb[m80]
        
        nbdx = 3600.0 * np.sqrt((bright[:,1]-nb8_cen[0])**2 * np.cos(nb8_cen[1]/180.0*np.pi)**2+(bright[:,2]-nb8_cen[1])**2) 

        if type == 'pm':
            ve = tangent_to_velocity(bright[:,4] * u.mas/u.yr)
            vn = tangent_to_velocity(bright[:,5] * u.mas/u.yr)
            nbpm = np.sqrt(ve**2 + vn**2)
            
            nbvd, nberr, nbvb = get_veldis(nbpm, nbdx)
            ax.errorbar(nbvb, nbvd, nberr, label=f'{label}', color=color, fmt=fmt)
            
        elif type == 'radial':
            nbvr = bright[:,6] # * u.km/u.s
            nbvd, nberr, nbvb = get_veldis(nbvr, nbdx)
            ax.errorbar(nbvb, nbvd, yerr=nberr, label=f'{label}', color=color, fmt=fmt)
        
    
    def plot_veldis_type(ax, t, label, **kwargs):
    
        vcut = (veldis.data_update['type'] == t) & (veldis.data_update['R'] > 10)
        verru = veldis.data_update['esig_u'][vcut]
        verrl = veldis.data_update['e_sigl'][vcut]
        verr = np.vstack((verrl, verru))
        ax.errorbar(veldis.data_update['R'][vcut], veldis.data_update['sig'][vcut], verr, label=label, **kwargs)
        
    fig, ax = plt.subplots()
    fig.tight_layout()
    ax.set(
        ylabel=r'Vel. disp. [km/s]',
        xlim=[10, 2500],
        xlabel=r'Radius [arcsec]'
    )
    ax.set_xscale("log")
    ax.grid()
    ax.yaxis.set_minor_locator(MultipleLocator(0.1))
    
    plot_veldis_type(ax, 1, 'Baumgardt \\& Hiker (2018) (3434 stars)', fmt='C1x')
    plot_veldis_type(ax, 4, 'Vasilev \\& Baumgardt (2021) (19734 stars)', fmt='C2x')
    plot_veldis_type(ax, 2, 'Libralato et al. (2022) (617 stars)', fmt='gx')
    
    ax.legend(loc='lower left', borderaxespad=0., borderpad=0.2)
    fig.savefig('veldisp.png')
    
    fig, axes = plt.subplots(nrows=2, figsize=(12/3, 24/3), sharex=True)
    fig.tight_layout()
    
    axes[0].set(
        ylabel=r'Vel. disp. [km/s]',
        xlim=[10, 2500]
    )
    axes[1].set(
        ylabel=r'Vel. disp. [km/s]',
        xlabel=r'Radius [arcsec]',
        xlim=[10, 2500]
    )
    
    for ax in axes:
        ax.set_xscale("log")
        ax.grid()
        ax.yaxis.set_minor_locator(MultipleLocator(0.1))
    
    plot_veldis_type(axes[0], 1, 'Baumgardt \\& Hiker (2018) (3434 stars)', fmt='C1x')
    plot_veldis_type(axes[1], 4, 'Vasilev \\& Baumgardt (2021) (19734 stars)', fmt='C2x')
    plot_veldis_type(axes[1], 2, 'Libralato et al. (2022) (617 stars)', fmt='gx')
    
    plot_veldis_nb(axes[0], 'NBODY (Bovy)', nbody, 'radial', '#595959')
    plot_veldis_nb(axes[0], 'NBODY (Irrgang)', nbody_irr, 'radial', '#5F9ED1')
    plot_veldis_nb(axes[1], 'NBODY (Bovy)', nbody, 'pm', '#595959')
    plot_veldis_nb(axes[1], 'NBODY (Irrgang)', nbody_irr, 'pm', '#5F9ED1')
        
    for ax in axes:
        ax.legend(loc='lower left', borderaxespad=0., borderpad=0.2)
    fig.savefig(save)

plot_velocity_dispersion('veldisp-nb.png')



In [ ]:
'''
from scipy.optimize import minimize

def vel_fac(f):
    m80 = (nbody[:,8] < 6) & (nbody[:,7] > 0.78) &  (nbody[:,7] < 0.9)
    bright = nbody[m80]
    dx = 3600.0 * np.sqrt((bright[:,1]-nb8_cen[0])**2 * np.cos(nb8_cen[1]/180.0*np.pi)**2+(bright[:,2]-nb8_cen[1])**2) 
    vr = bright[:,6]
    
    def sig(r):
        cut = dx <= r
        vm = np.mean(vr[cut])
        outlier = (np.abs(dx - vm) > 25)
        bds = ~outlier & cut
        n = np.sum(bds)
        return np.sqrt(1/n * np.sum((dx[bds] - vm)**2.0))
    
    nbs, _, _ = get_veldis(vr, dx, veldis.data_update['R'])
    cut = ~np.isnan(nbs)
    
    #nbs = np.array([sig(r) for r in veldis.data_update['R']])
    return np.sum((f*nbs[cut] - veldis.data_update['sig'][cut])**2 / (nbs[cut]**2 + veldis.data_update['sig'][cut]**2))


print(minimize(vel_fac, 5))
x = np.linspace(0.9, 1.2)
y = np.array([vel_fac(f) for f in x])
plt.plot(x, y)

'''
pass

In [ ]:

#lagr_ev = np.loadtxt('2023-11-05/fort.32')
lagr_ev = np.loadtxt('2024-04-07/fort.32')
FLAGR=[0.01, 0.02, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5, 0.625,0.75,0.99]

def plot_lagr(lagr, title, xrange):
    r = 300
    r = np.amax(lagr[:,1])
    fig, ax = plt.subplots(ncols=1, nrows=1)
    fig.tight_layout()

    ax.set_title(title)
    ax.set_xlabel(r'T (Myr)')
    ax.set_ylabel(r'Radius')
    #x.set_xscale("log")
    ax.set_xlim([-4000, 0])
    ax.grid()

    for i,f in enumerate(FLAGR):
        b = lagr[:,1] <= r
        ax.plot(lagr[:,1][b]-13500, lagr[:,i+2][b], label='{} M'.format(f))
    ax.legend()
        
plot_lagr(lagr_ev, "Lag. Radii - 4 Gyr (evolution)", [9500, 13500])
#plot_lagr(lagr_4gyr, "Lag. Radii - 4 Gyr (no evolution)", [0, 4000])

In [ ]:
#boundm = np.loadtxt('2023-11-05/boundm')

boundm_bovy = np.loadtxt('2024-04-07/boundm')
boundm_irr = np.loadtxt('2024-04-28/boundm')
boundm_bovy[0,0] = 9500
boundm_irr[0,0] = 9500
    
fig, ax = plt.subplots()
fig.tight_layout()

ax.set_xlim(-3000, 0)
ax.set_ylabel('$M/M_0$, $N/N_0$')
ax.set_xlabel('Time [Myr]')
ax.grid()
ax.plot(boundm_bovy[:,0] - 13500, boundm_bovy[:,2], color='#595959', label='Mass (Bovy) [$M/M_0$]')
ax.plot(boundm_bovy[:,0] - 13500, boundm_bovy[:,3], '+', color='#595959', label='Stars (Bovy) [$N/N_0$]')

ax.plot(boundm_irr[:,0] - 13500, boundm_irr[:,2], color='#5F9ED1', label='Mass (Irrgang) [$M/M_0$]')
ax.plot(boundm_irr[:,0] - 13500, boundm_irr[:,3], '+', color='#5F9ED1', label='Stars (Irrgang) [$N/N_0$]')
ax.legend()
fig.savefig('boundm.png')


fig, ax = plt.subplots()
fig.tight_layout()
ax.set_ylabel('Radius [pc]')
ax.set_xlabel('Time [Myr]')

def plot_r(boundm, ax, fmt, label):
    x = boundm[:,0] - 13500
    y = boundm[:,1]
    coef = np.polyfit(x,y,1)
    poly1d_fn = np.poly1d(coef) 

    ax.plot(x, y, '-', x, poly1d_fn(x), fmt, label=label)

plot_r(boundm_bovy, ax, '--k', 'Bovy')
plot_r(boundm_irr, ax, '--g', 'Irrgang')
ax.legend()

fig.savefig('r.png')




In [ ]:


'''
def plot_bh(ax, nb, label, fmt, color):
    bh = nb[:,8] >= 13
    print(np.sum(bh))
    d = 3600.0 * np.sqrt((nb[:,1][bh]-orbits.RA)**2 * np.cos(orbits.DEC/180.0*np.pi)**2+(nb[:,2][bh]-orbits.DEC)**2) 
    
    logmin = np.log10(np.amin(d))
    logmax = np.log10(np.amax(d))
    logbins = np.logspace(logmin, logmax, 10)
    counts, bins = np.histogram(d, bins=logbins)
    cut = counts > 0
    
    # Divide the sum in each bin by the area of the annulus
    r_b = bins[:-1]
    area = np.array([
      np.pi * (bins[i+1]**2 - r**2)
      for i,r in enumerate(r_b)
    ])[cut]
    
    n_area = counts[cut] #/ area 
    nsq_area = np.sqrt(counts[cut])# / area 
    #y = -2.5 * np.log10(n_area)
    #error = -2.5 * np.log10(n_area+nsq_area) - y
    y = n_area
    error = n_area + nsq_area - y
    ax.errorbar(r_b[cut], y, yerr=np.abs(error), label=label, lw=1, elinewidth=1, fmt=fmt, color=color)

fig, ax = plt.subplots()
fig.tight_layout()
ax.set_ylabel('$N_{BH}$')
ax.set_xlabel('Radius [arcsec]')
ax.set_xscale("log")
ax.grid()

plot_bh(ax, nbody, 'Bovy', 'x', '#595959')
'''
nbform = np.loadtxt('2024-04-07/ngc6397.nbform')
nborb = np.loadtxt('2024-04-07/orbit')

#d = 3600.0 * np.sqrt((nbody[:,1]-orbits.RA)**2 * np.cos(orbits.DEC/180.0*np.pi)**2+(nbody[:,2]-orbits.DEC)**2) 
d = np.sqrt(
    (nbform[:,1]-nborb[-1,1])**2 +
    (nbform[:,2]-nborb[-1,2])**2 +
    (nbform[:,3]-nborb[-1,3])**2
)

he = nbform[:,-1] == 10
co = nbform[:,-1] == 11
ne = nbform[:,-1] == 12
ns = nbform[:,-1] == 13
bh = nbform[:,-1] == 14

remnant = he | co | ne | ns | bh
close = d < 70
print(np.amin(d), np.amax(d))
print(np.sum(remnant & close))
print(np.sum(nbform[:,0][remnant&close]) / np.sum(nbform[:,0][close]))
print(np.sum(nbform[:,0][remnant&close]))

print(np.sum((he|co|ne)&close))
print(np.sum(ns & close))
print(np.sum(bh & close))

fig, ax = plt.subplots()
fig.tight_layout()
ax.set_ylabel('$M(r) [M_{\\odot}]$')
ax.set_xlabel('Radius [pc]')
ax.set_xscale("log")
ax.set_yscale("log")
ax.grid()

data = {
    #'[He] WD': he,
    '[CO] WD': co,
    '[ONeMg] WD': ne,
    'NS': ns,
    'BH': bh
}

for label, select in data.items():
    bins = np.logspace(0.9, 1.85, num=50)
    m = []
    for b in bins:
        inside = d < b
        m.append(np.sum(nbform[:,0][select&inside]))
        
    #n, x, _ = plt.hist(d[select&close], bins=10, cumulative=True, weights=nbody[:,7][select&close])
    ax.plot(bins, m, '--', label=label)

ax.legend(loc='upper left', borderaxespad=0., borderpad=0.2)

fig.savefig('compact_mass.png')